In [4]:
import sys
sys.path.insert(0, '../zdrojaky')

import numpy as np
import matplotlib.pylab as plt
import nig
np.set_printoptions(precision=2)

# Přednáška 2: Sekvenční odhad lineárních modelů, predikce

Uvažujme pozorovatelný náhodný proces $\{Y_t|X_t; t=1,2,\ldots\}$ s nezávislými stejně rozdělenými realizacemi $y_1, y_2,\ldots$ determinovanými známou pozorovanou veličinou $X_t$, např. regresorem. Ten může a nemusí existovat, o tom více později. Naším cílem nechť je tento proces modelovat prostřednictvím vhodného pravděpodobnostního modelu $f(y_t|x_t, \theta)$, kde $\theta$ je neznámý parametr, jehož odhad je samozřejmě pro kvalitu modelu rozhodující. Uvažujme dále, že toto modelování chceme dělat online, např. pro účely předpovědi (počasí, směnných kurzů, polohy letícího cíle aj.).

Existuje více způsobů, jak k modelování přistoupit. Například:
- počítat periodicky odhad na **datovém okně**. To může zahrnovat všechna data od počátku $t=1$ až po okamžik současný. Nevýhodou tohoto způsobu je jeho velká náročnost na paměť a postupně rostoucí výpočetní náročnost, vyžadující rychlé zpracování i extrémního množství dat v krátkém čase.
- periodicky odhadovat $\theta$ na **plovoucím datovém okně**. Jde o jeden z velmi častých přístupů, kdy sice používáme ta samá data nějakou dobu dokola, ale jejich množství je omezené. Můžeme například počítat z posledních 100 dat a předchozí data zahazovat.
- plně sekvenční odhad, kdy ke stávající informaci přidáváme pouze nejnovější data. Pokud je model snadno upočitatelný a data nemění rozdělení, je tento postup obzvlášť výhodný.

## Příklady modelů

## Bayesovské odhadování - malé repetitorium

Označme $y_t$ pozorovanou náhodnou veličinu (data, měření) v diskrétních časových okamžicích $t=0, 1,2,\ldots$, a vektor jejích hodnot $y_{0,t-1} = [y_0, y_1, \ldots, y_{t-1}]$. Budiž jednotlivé $y_t$ nezávislé a stejně rozdělené. Nechť je $y_t$ determinováno známou veličinou $x_t$ (např. regresorem) a konstatním parametrem $\theta$. Označme rovněž $x_{0,t-1} = [x_0, x_1, \ldots, y_{x-1}]$. 

Uvažujeme dále, že apriorní hustota $\pi(\theta|x_0, y_0)$ nese nějakou "vhodnou" znalost, např. z minulosti, nebo je neinformativní. Veličiny $x_0$ a $y_0$ mají roli "pseudo-dat$ (pseudopozorování, pseudoměření).

> **Věta (Bayesova)**
>
> Nechť $f(y_t|x_t, \theta)$ je hustota pravděpodobnosti $y_t|x_t,\theta$. Dále nechť $\pi(\theta|y_{0:t-1}, x_{0:t-1})$ je apriorní hustota pro $\theta$. Aposteriorní hustota pro jednokrokový update má tvar
>
$$
\begin{aligned}
\pi(\theta|y_{0:t}, x_{0,t}) 
&= 
\frac
{f(y_t|x_t, \theta) \pi(\theta|x_{0:t-1}, y_{0:t-1})}
{\int f(y_t|x_t, \theta) \pi(\theta|x_{0:t-1}, y_{0:t-1})d\theta} \\
&=
\frac
{f(y_t|x_t, \theta) \pi(\theta|x_{0:t-1}, y_{0:t-1})}
{f(y_t|x_t)} \\
&\propto
f(y_t|x_t, \theta) \pi(\theta|x_{0:t-1}, y_{0:t-1}).
\end{aligned}
$$

Poslední řádek bude náš nejoblíbenější, neboť vyjadřuje bayesovský update v jednoduchém zápisu - bez normalizující hustoty ve jmenovateli. Ta je totiž nezávislá na odhadovaném parametru $\theta$. Normalizaci si (*teoreticky*) snadno zařídíme kdykoliv.

Odvoďme si nyní bayesovský update pro více dat současně:
$$
\begin{aligned}
\pi(\theta|y_{0:1}, x_{0:1})
&\propto
f(y_1|x_1, \theta) \pi(\theta|x_{0}, y_{0}) \\
\pi(\theta|y_{0:2}, x_{0:2})
&\propto
f(y_2|x_2, \theta) \pi(\theta|x_{0:1}, y_{0:1}) \\
&\propto \pi(\theta|x_{0}, y_{0}) f(y_1|x_1, \theta) f(y_2|x_2, \theta) \\
&\vdots \\
\pi(\theta|y_{0:t}, x_{0:t})
&\propto
\pi(\theta|y_{0}, x_{0})
\prod_{\tau = 1}^{t}
f(y_{\tau}|x_{\tau}, \theta) \\
&\propto
\pi(\theta|y_{0:\tau-1}, x_{0:\tau-1})
\prod_{\tau = \widetilde{\tau}}^{t}
f(y_{\widetilde\tau}|x_{\widetilde\tau}, \theta),
\end{aligned}
$$
kde předposlední řádek říká, že můžeme pronásobit modely a pak teprve updatovat počáteční apriorno a poslední řádek říká, že totéž můžeme udělat v kterémkoliv čase. Závěr: sekvenční update po jednom datu je stejný, jako update více daty najednou.

** Úkoly **
- jak si zařídíme normalizaci v posledním řádku uvedené Bayesovy věty?
- uvědomte si, jak vypadá aposteriorno. Jaké musí mít vlastnosti. A co když vezmeme jako model nějakou distribuci, např. beta a jako apriorno např. také beta? Co můžeme o vzniklém aposteriornu říci?

## Sekvenční odhad

Výše jsme si ukázali, že teoreticky je sekvenční bayesovský update jednoduchá záležitost. Použijeme apriorní distribuci, přidáme nové pozorování, dostaneme aposteriorní a tu opět použijeme jako apriorní pro další pozorování:

$$
\pi(\theta|x_{0}, y_{0}) \xrightarrow[\text{Bayes}]{x_1, y_1}
\pi(\theta|x_{0:1}, y_{0:1}) \xrightarrow[\text{Bayes}]{x_2, y_2}
\pi(\theta|x_{0:2}, y_{0:2}) \rightarrow
\cdots \xrightarrow[\text{Bayes}]{x_t, y_t}
\pi(\theta|x_{0:t}, y_{0:t}) \rightarrow
\cdots
$$

Připomeňme dále, že jako bodový odhad $\theta$ můžeme použít například
- střední hodnotu $\mathbb{E}[\theta]$, přesněji $\mathbb{E}[\theta|x_{0:t}, y_{0:t}]$, nebo 
- modus (maximum hustoty psti) - tzv. maximum aposteriori (MAP) odhad, nebo 
- medián (prostřední hodnotu).

Jak moc tomuto odhadu věříme, neboli jak mnoho informace aposteriorní distribuce obsahuje, nám udává například rozptyl, $\text{var}\theta$.

**
Ač se to ze vzorců výše možná nezdá, zásadním problémem bayesovského odhadu (modelování) je odvození aposteriorní distribuce a jejích vlastností. A se sekvenčním odhadem se tento problém ještě umocňuje. Pokud po prvním updatu není aposteriorno některou z "běžných" distribucí, pak s dalším updatem je aposteriorno ještě komplikovanější.
**

**
Existuje ale případ, kdy výpočty půjdou velmi pohodlně. K tomu si musíme zavést exponenciální třídu distribucí a konjugovaná apriorna.
**

> **Definice (exponenciální třída distribucí)**
>
> Uvažujme náhodnou veličinu $y$ podmíněnou veličinou $x$ a parametrem $\theta$. Exponenciální třída distribucí obsahuje distribuce s hustotou pravděpodobnosti ve tvaru
> $$
f(y|x, \theta) = h(y, x) g(\theta) \exp \left[ \eta^{\intercal} T(y,x) \right],
$$
>
> kde $\eta \equiv \eta(\theta)$ je přirozený parametr, $T(y,x)$ je suficientní statistika fixního rozměru, $h(y,x)$ je známá funkce a $g(\theta)$ je normalizační funkce. Platí-li $\eta(\theta)=\theta$ je třída kanonická.


> **Definice (konjugovaná apriorní distribuce)**
>
> Nechť $y|x, \theta$ má rozdělení z exponenciální třídy distribucí. Říkáme, že apriorní distribuce $\theta$ s hyperparametry $\xi$ a $\nu$ je k němu konjugovaná, pokud její hustota pravděpodobnosti má tvar
$$
        \pi(\theta) = q(\xi, \nu) g(\theta)^{\nu} \exp \left[ \eta^{\intercal} \xi \right],
$$
>
>kde $\xi$ má stejný rozměr jako $T(y,x)$, $\nu\in\mathbb{R}^{+}$ a $q(\xi,\nu)$ je známá funkce. Funkce $g(\theta)$ je stejná jako normalizační funkce v hustotě pro $y|x, \theta$.

$$
\begin{aligned}
    \xi_{t} &= \xi_{t-1} + T(y_{t},x_{t}), \\
    \nu_{t} &= \nu_{t-1} + 1.
\end{aligned}
$$
$$
\begin{aligned}
    \xi_{t} &= \xi_{\tau-1} + \sum_{\widetilde{\tau}=\tau}^{t} T(y_{\widetilde{\tau}},x_{\widetilde{\tau}}),\\
    \nu_{t} &= \nu_{\tau-1} + t - \tau+1.
\end{aligned}
$$

### Příklady

#### Normální model 
Uvažujme $X \sim \mathcal{N}(\mu, \sigma^2)$ kde $\sigma^2$ je známý rozptyl. Potom
$$
f(x|\mu, \sigma^2) 
=
\frac{1}{\sqrt{2\pi\sigma^2}} \exp 
\left(
\frac{-1}{2\sigma^2} (x - \mu)^2
\right)
=
\underbrace{\frac{1}{\sqrt{2\pi\sigma^2}} e^{-\frac{x^2}{2\sigma^2}}}_{h(x)}
\times
\underbrace{e^{-\frac{\mu^2}{2\sigma^2}}}_{g(\theta)\equiv g(\mu)}
\exp 
\underbrace{
\left(
\frac{\mu}{\sigma^2} x
\right)
}_{\eta \cdot T(x)},
$$
kde tedy přirozený parametr $\eta \equiv \eta(\mu) = \mu/\sigma^2$ a suficientní statistika $T(x) = x$. Podotkněme, že $\sigma^2$ by mohlo být rovněž pod $x$ v suficientní statistice. Tvar totiž není jednoznačný.

## Regresní model

## Bayesovský regresní model

### Model
Uvažujme znovu lineární model pro skalární reálné $y_t$ determinované regresním vektorem $x_t\in\mathbb{R}^p$ a vektorem regresních koeficientů $\beta\in\mathbb{R}^p$ ve tvaru
$$
\begin{aligned}
y_t &= \beta^\intercal x_t + \varepsilon_t, \\
\varepsilon_t &\sim \mathcal{N}(0, \sigma^2) \qquad \text{iid}.
\end{aligned}
$$

Takový model je tedy ekvivalentně $y_t\sim\mathcal{N}(\beta^\intercal x_t, \sigma^2)$ a má hustotu
$$
\begin{aligned}
    f(y_{t}|z_{t}, \beta, \sigma^{2}) 
    &= \frac{(\sigma^{2})^{-\frac{1}{2}}}{\sqrt{2\pi}}
       \exp
       \left\{ 
           -\frac{1}{2\sigma^{2}} (y_{t} - \beta^{\intercal}z_{t})^{2} 
       \right\} \notag \\
    &= \frac{(\sigma^{2})^{-\frac{1}{2}}}{\sqrt{2\pi}}
       \exp
       \Bigg\{ 
           \text{Tr}
           \bigg( 
               \underbrace{
                   -\frac{1}{2\sigma^{2}}
                   \begin{bmatrix}
                       1 \\ -\beta
                   \end{bmatrix}
                   \begin{bmatrix}
                       1 \\ -\beta
                   \end{bmatrix}^{\intercal}
               }_{\eta}
               \underbrace{
                   \begin{bmatrix}
                       y_{t} \\ z_{t}
                   \end{bmatrix}
                   \begin{bmatrix}
                       y_{t} \\ z_{t}
                   \end{bmatrix}^{\intercal}
               }_{T(y_{t}, z_{t})}    
           \bigg)
       \Bigg\}.
\end{aligned}
$$

### Apriorní distribuce
Jelikož neznáme ani regresní koeficienty ve vektoru $\beta$, ani varianci šumu $\sigma^2$, budeme je odhadovat. Vhodná konjugovaná apriorní distribuce je **normální inverzní-gama** distribuce,

$$
\beta, \sigma^{2} 
\sim \mathcal{N}i\mathcal{G}(m_{t-1}, V_{t-1}, a_{t-1}, b_{t-1})
= \mathcal{N}(m_{t-1}, \sigma^{2} V_{t-1}) \times i\mathcal{G}(a_{t-1}, b_{t-1}),
$$

s reálnými hyperparametry $a_{t-1}>0$ a $b_{t-1}>0$, vektorem středních hodnot $m_{t-1}\in\mathbb{R}^{p}$ a škálovací maticí $V_{t-1}^{-1}$ odpovídající dimenze. Jen pro zajímavost si ukažme hustotu pravděpodobnosti takové distribuce:

$$
\pi(\beta, \sigma^{2}|\cdot)
    = \frac{b^{a_{t-1}} (\sigma^{2})^{-(a_{t-1}+1+\frac{p}{2})}}{\sqrt{2\pi}|V_{t-1}|^{\frac{1}{2}}\Gamma(a_{t-1})}
       \exp
       \Bigg\{ 
           -\frac{1}{2\sigma^{2}}
           \bigg[ 
           b_{t-1} + 
               \text{Tr}
               \bigg( 
                       \begin{bmatrix}
                           1 \\ -\beta
                       \end{bmatrix}^{\intercal}
                       \begin{bmatrix}
                           1 \\ -\beta
                       \end{bmatrix}
                       \begin{bmatrix}
                           m_{t-1}^{\intercal} \\ I 
                       \end{bmatrix}
                       V_{t-1}^{-1}
                       \begin{bmatrix}
                           m_{t-1}^{\intercal} \\ I 
                       \end{bmatrix}^{\intercal}
                \bigg)
            \bigg]
       \Bigg\}.
$$

Porovnáním s hustotou modelu lze dovodit, že pro nás je důležité určit hyperparametry $\xi_{t-1}$ a $\nu_{t-1}$, jež zastoupí naše $a_{t-1}, b_{t-1}, m_{t-1}$ a $V_{t-1}$. Ty jsou

$$
\begin{aligned}
    \xi_{t-1} 
    &=
    \begin{bmatrix}
        m_{t-1}^{\intercal} V_{t-1}^{-1} m_{t-1} + 2b_{t-1} & m_{t-1}^{\intercal} V_{t-1}^{-1} \\
        V_{t-1}^{-1}m_{t-1} & V_{t-1}^{-1}
    \end{bmatrix} \\
    &=
    \begin{bmatrix}
        \xi_{t-1}^{[11]} & \xi_{t-1}^{[12]} \\
        \xi_{t-1}^{[21]} & \xi_{t-1}^{[22]}
    \end{bmatrix}, \\
\nu_{t-1} &= 2a_{t-1}.
\end{aligned}
$$

### Bayesovský update
Bayesovský update - součet hyperparametru se suficientní statistikou - nám pak odhalí aposteriorní distribuce a povětšinou snadnou algebrou pak dostaneme

$$
\begin{aligned}
    V_{t} &= \left( V_{t-1}^{-1} + x_{t}x_{t}^{\intercal} \right)^{-1}
           = V_{t-1} - \frac{V_{t-1} x_{t}x_{t}^{\intercal} V_{t-1}}{1+x_{t}^{\intercal} V_{t-1} x_{t}}= \left(\xi_{t}^{[22]}\right)^{-1}, \\ 
    m_{t} &= V_{t}(V_{t-1}^{-1}m_{t-1} + y_{t}x_{t}) = \left(\xi_{t}^{[22]}\right)^{-1} \xi_{t}^{[21]}, \\
    a_{t} &= a_{t-1} + \frac{1}{2} = \frac{1}{2}(\nu_{t-1} + 1) = \frac{1}{2}\nu_{t}, \label{eq:nig-update} \\
    b_{t} &= b_{t-1} + \frac{1}{2} \left(-m_{t}^{\intercal}V_{t}^{-1}m_{t} + m_{t-1}^{\intercal} V_{t-1}^{-1}m_{t-1} + y_{t}^{2} \right) \\
    &= \frac{1}{2}\left[\xi_{t}^{[11]} - \xi_{t}^{[12]}\left( \xi_{t}^{[22]} \right)^{-1} \left( \xi_{t}^{[12]} \right)^{\intercal}\right], \notag
\end{aligned}
$$

Není bez zajímavosti, že tento přístup k nalezení aposteriorních hyperparametrů je nemálo snadnější, než jejich odvozování z původní podoby, které je ale v literatuře běžné.

Připomeňme, co zaznělo výše:
$$
\beta, \sigma^{2} = \mathcal{N}(m_{t-1}, \sigma^{2} V_{t-1}) \times i\mathcal{G}(a_{t-1}, b_{t-1}).
$$

Z toho víme, že aposteriorním bodovým odhadem $\beta$ je $m_t$. Odhad $\sigma^2$, označený $\hat{\sigma}^2$ dostaneme pohledem do vlastností [inverzní gama distribuce](https://en.wikipedia.org/wiki/Inverse-gamma_distribution),
$$
    \hat{\sigma}^{2} = \frac{b_{t}}{a_{t}-1}  \quad\text{a}\quad
    \operatorname{var}(\sigma^{2}|\cdot) = \frac{b_{t}^{2}}{(a_{t}-1)^{2}(a_{t}-2)}.
$$

Kdybychom chtěli jít do detailu, marginální distribuce pro $\beta$ je [Studentova t distribuce](https://en.wikipedia.org/wiki/Student%27s_t-distribution#Non-standardized_Student%27s_t-distribution) s $2a_t$ stupni volnosti, centrovaná ve zmíněném $m_t$ a se škálovací maticí $\frac{a_t}{b_t}V_t$.

### Predikce
Předpokládejme, že se ptáme na hodnotu $y'$ pro zadané $x'$, například budoucí hodnotu. Jako bayesovci na to půjdeme přes prediktivní hustotu,
$$
f(y'|y_{0:t},x_{0:t},x') = \int f(y'|x', \beta, \sigma^{2}) \pi(\beta, \sigma|y_{0:t}, x_{0:t}) \mathrm{d}\beta \mathrm{d}\sigma^{2}.
$$

To je opět [Studentova t distribuce](https://en.wikipedia.org/wiki/Student%27s_t-distribution#Non-standardized_Student%27s_t-distribution)
$$
y'|y_{0:t}, x_{0:t}, x' \sim t_{2a_{t}}\left(m_{t}^{\intercal}x', \frac{b_{t}}{a_{t}} \left(1 + (x')^{\intercal}V_{t}x'\right) \right).
$$